In [10]:
import pandas as pd
from re import search
import music21 as m21
import math
import numpy as np
from random import seed
from random import randint

class pieceTypes:
    def parse(YCAC_Excel, name):
        list_of_column_headers = []
        for column in YCAC_Excel.columns:
            list_of_column_headers.append(column)
       
        listOfPieces = []
        for index, row in YCAC_Excel.iterrows():
            if search(name,str(row[7]).lower()) and row[7] not in listOfPieces:
                listOfPieces.append(row[7])
        return listOfPieces

    def extractPieces(YCAC_Excel, name):
        listOfPieces = pieceTypes.parse(YCAC_Excel, name)
        print("Excel parsed and pieces extracted")
        
        separatedPieces = []
        
        for piece in listOfPieces:
            separatedPieces.append([])
            
        for index, row in YCAC_Excel.iterrows():
            if row[7] in listOfPieces:
                for index2, item in enumerate(listOfPieces):
                    if item==row[7]:
                        separatedPieces[index2].append([row[0],row[1]])
        return separatedPieces
    
    def m21FormattedChords(YCAC_Excel, name):
        separatedPieces = pieceTypes.extractPieces(YCAC_Excel, name)
        print("Chords ready to be formatted")
        
        m21PieceFormatted = []
        for pieceIndex, eachPiece in enumerate(separatedPieces):
            
            m21Formatted = []
            for eachChord in eachPiece:
                newChord = []
                for thing in eachChord[1].replace("<","").replace(">","").split():
                    if len(thing)>7:
                        continue
                    else:
                        newChord.append(thing)
                m21Formatted.append([eachChord[0], m21.chord.Chord(newChord)])
            m21PieceFormatted.append(m21Formatted)
        
        return m21PieceFormatted

    def rhythmicWeighting(YCAC_Excel, name, duration, windowForRegister, register):
        m21FormattedPiece = pieceTypes.m21FormattedChords(YCAC_Excel, name)
        print("weighting process starting")
        weightedChords = []
        totalWeighting = []
        for pieceIndex, eachPiece in enumerate(m21FormattedPiece):
            weightedChords.append([])
            totalWeighting.append([])
            for chordIndex, chord in enumerate(eachPiece):
    #             print(chord)
                newEncoding = []
                aggregateEncoding = []
                midiForm = [p.midi for p in chord[1].pitches]
                for thing in chord:
                    newEncoding.append(thing)
                    aggregateEncoding.append(thing)
                cardinality = len(midiForm)
                logCard = math.log(cardinality, 2)+.5
                newEncoding.append(logCard)
            #     print(newEncoding)

            #   Duration = duration * 2
                if chordIndex+1 < len(eachPiece):
                    currentDuration = eachPiece[chordIndex+1][0] - chord[0]
                    durationWeight = currentDuration*duration
                    newEncoding.append(durationWeight)

            #   registral weighting
                lowestPitch = min(midiForm)
                registerCheck = 0

                if chordIndex+windowForRegister < len(eachPiece):
                    for window in range(windowForRegister+1):
                        if chordIndex - (window) >= 0:
                            chordBack = chordIndex-window
                            if chordBack != chordIndex:
                                chordBackMidiForm = [p.midi for p in eachPiece[chordBack][1].pitches]
                                lowestPitchBack = min(chordBackMidiForm)
                                if lowestPitch - lowestPitchBack >= 0:
                                    registerCheck+=1
                        chordForward = chordIndex+window
                        if chordForward!= chordIndex:
                            chordForwardMidiForm = [p.midi for p in eachPiece[chordForward][1].pitches]
                            lowestPitchForward = min(chordForwardMidiForm)
                            if lowestPitch - lowestPitchForward > 0:
                                registerCheck+=1
                if registerCheck > 0:
                    newEncoding.append(0)
                if registerCheck == 0:
                    newEncoding.append(register)

    #           Non grouped weightings
                weightedChords[pieceIndex].append(newEncoding)

                totalWeight = 0
                for indexNew, thing in enumerate(newEncoding):
                    if indexNew>1:
                        totalWeight+=thing
                aggregateEncoding.append(totalWeight)

                totalWeighting[pieceIndex].append(aggregateEncoding)
        return weightedChords, totalWeighting
    
    def gridifyM21Chords(YCAC_Excel, name, division):
        durationWeight = int(input('Assign duration weight'))
        windowSize = int(input('Assign window for register check'))
        windowWeight = int(input('Assign weight for lowest pitch in window'))
        weightedChords = pieceTypes.rhythmicWeighting(YCAC_Excel, name,durationWeight,windowSize,windowWeight)
        print("starting grids")
        gridifiedPieces = []
        
#         weightedChord[1] is total weighting
        for piece in weightedChords[1]:
            pieceGrid = []
            pieceGrid = [0 for i in range(int(piece[-1][0]/division)+1)]
            for chord in piece:
                pieceGrid[int(chord[0]/division)] = chord[2]
            gridifiedPieces.append(pieceGrid)   
        return gridifiedPieces



# seed just makes sure that it'll be the same random numbers as last time
def extractPieceSlicesAndDFTs(pieceGrids, numberOfSlices, lengthOfSlice):
    seed(1)
    listOfPieceSlices = []
    listOfDFTs = []
    for pieces in range(0,numberOfSlices):
        randomPiece = randint(0,len(pieceGrids)-1)
        piece = pieceGrids[randomPiece]
    #   96 slice length is 12 quarter-note beats
        sliceLength = lengthOfSlice
        randomStartingPosition = randint(0,len(piece)-(sliceLength+1))
        pieceSlice = piece[randomStartingPosition:randomStartingPosition+sliceLength+1]       
        listOfPieceSlices.append(pieceSlice)
        
    return listOfPieceSlices

# Used in dftSlices
def normalizeDFTSlices(dftMagnitudeArray):
    normalizedDFTs = []
    for eachPiece in dftMagnitudeArray:
        tempArray = []
        weighting = eachPiece[0]*eachPiece[0]
        for component in eachPiece:
            componentNormalized = (component*component)/weighting
            tempArray.append(componentNormalized)
        normalizedDFTs.append(tempArray)
    return normalizedDFTs

def dftSlices(weighted_data):
    DFTArray = []
    magnitudeArray = []
    phaseArray = []
    for pieceSlice in weighted_data:
        zeroPaddedSlice = pieceSlice + [0 for i in pieceSlice]
        DFTArray.append(np.fft.fft(zeroPaddedSlice)) 
    for eachArray in DFTArray:
        magnitudeArrayGrabber = np.absolute(eachArray)
        magnitudeArray.append(magnitudeArrayGrabber)
        phaseArrayGrabber = np.angle(eachArray)
        phaseArray.append(phaseArrayGrabber)
        
    normedMagnitudes = normalizeDFTSlices(magnitudeArray)
    
    return magnitudeArray, normedMagnitudes, phaseArray, DFTArray


def dataFrameWithTargets(pieceSlices, targetName):
    dataFrameSlices = pd.DataFrame(pieceSlices)
    pieceCategory = [targetName for i in dataFrameSlices[0]]
    dataFrameSlices["target"] = pieceCategory
    return dataFrameSlices

import pandas as pd
def oneHotCategories(targets):
    oneHotSeries = []
    for row, index in targets.iteritems():
        oneHot = [0, 0, 0, 0]
        oneHot[index]+=1
        oneHotSeries.append(oneHot)
    newTargets = pd.Series(oneHotSeries)
    return oneHotSeries, newTargets